# $$ 0.\quad Data \quad Input $$

In [1]:
import sys
from gurobipy import *
from myMatrixLpSolver import lp_optimize
import numpy as np
f = open('dataLR.txt', 'r')
N=10000
C=10
x = [[0 for j in range(C)] for i in range(N)] 
y = [0 for i in range(N)] 
# Loop over lines and extract variables of interest
i=-1
for line in f:
		i=i+1
		line = line.strip()
		columns = line.split(',')
		y[i]=float(columns[0])
		for j in range(0,10):
			x[i][j] = float(columns[j+1])

f.close()

x = np.array(x)
y = np.array(y)

In [7]:
x.shape, y.shape

((10000, 10), (10000,))

# $$ 1.\quad min\quad max\quad|y_i −w_Tx_i −b|$$

In [10]:
#Primal
# Put model data into matrices/vectors and call lp_optimize
# Questions 1(a)
rows = 20000
cols = 12
#Construct A
c = np.zeros(12)
c[0] = 1
obj_sense = GRB.MINIMIZE
sense = [GRB.GREATER_EQUAL for i in range(rows)]
lb = [-GRB.INFINITY for i in range(cols)]
ub = [GRB.INFINITY for i in range(cols)]
vtype = [GRB.CONTINUOUS for i in range(cols)]
solution = np.zeros(12)
rhs = [y[i] if i<10000 else -y[i-10000] for i in range(rows)]
A = np.array(x)
A = np.append(A, -A, axis = 0)
one_array = np.ones((rows, 1))
A = np.column_stack((one_array, A))
one_array[10000:] = -one_array[10000:]
A = np.column_stack((A, one_array))


lp_optimize(rows, cols, c, obj_sense, A, sense, rhs, lb, ub, vtype,
                   solution)

print("omega:")
print(solution[2:])

Optimize a model with 20000 rows, 12 columns and 237130 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 2e+04]
Presolve time: 0.08s
Presolved: 12 rows, 20000 columns, 237130 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   4.000000e+00   6.713220e+09      0s
     242    5.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 242 iterations and 0.12 seconds
Optimal objective  5.000000000e+01
omega:
[-2.66171867e-13 -1.20000000e+01  2.80000000e+01 -2.50000000e+01
  6.00000000e+00  2.10000000e+01  6.40000000e+01  3.60000000e+01
  1.00000000e+02  7.70000000e+01]


In [2]:
#Dual
rows = 12
cols = 20000
c = np.append(y,-y)
obj_sense = GRB.MAXIMIZE
rhs = [0]*11+[1]
lb = [0]*20000
ub = [GRB.INFINITY]*20000
vtype = [GRB.CONTINUOUS]*20000
A = (np.column_stack((x, -np.ones(10000)))).T
A = np.column_stack((A, -A))
A = np.row_stack((A, np.ones(20000)))
sense = [GRB.EQUAL]*11+[GRB.LESS_EQUAL]
solution = [0]*20000
lp_optimize(rows, cols, c, obj_sense, A, sense, rhs, lb, ub, vtype,
                   solution)

Academic license - for non-commercial use only
Optimize a model with 12 rows, 20000 columns and 237130 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [6e+00, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.14s
Presolved: 12 rows, 20000 columns, 237130 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.600e+01
 Factor NZ  : 7.800e+01 (roughly 8 MBytes of memory)
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.06128439e-11 -0.00000000e+00  6.71e+02 1.25e+01  2.24e+01     0s
   1  -9.71320339e-02  2.69592814e+03  5.76e+01 9.09e-13  2.08e+00     0s
   2  -3.43338972e-03  2.35891883e+03  1.44e-13 8.53e-13  1.18e-01     0s
   3   4.17163341e-03  1.5959470

True

# $$2.\quad min \quad Σ |y_i −w_Tx_i −b|$$

In [12]:
# Primal
cols = 20011
lb = [0]*20000+[-GRB.INFINITY]*11
ub = [GRB.INFINITY]*20011
rows = 10000
rhs = y
A = np.column_stack((np.eye(10000),-np.eye(10000),x,-np.ones(10000)))
c = [1]*20000+[0]*11
sense = [GRB.EQUAL]*10000
obj_sense = GRB.MINIMIZE
vtype = [GRB.CONTINUOUS]*20011
solution = np.zeros(20011)
lp_optimize(rows, cols, c, obj_sense, A, sense, rhs, lb, ub, vtype,
                   solution)
print("omega and b:")
print(solution[20000:len(solution)])

Optimize a model with 10000 rows, 20011 columns and 128565 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 2e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.09s
Presolved: 10000 rows, 20011 columns, 128565 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 11
 Free vars  : 11
 AA' NZ     : 1.086e+05
 Factor NZ  : 1.186e+05 (roughly 13 MBytes of memory)
 Factor Ops : 1.408e+06 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.13 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 30 iterations and 0.13 seconds
Optimal objective  1.000000000e+03
omega and b:
[ -9.   0. -12.  28. -25.   6.  21.  64.  36. 100. -27.]


In [17]:
#Dual
rows = 11
cols = 10000
c = y
obj_sense = GRB.MAXIMIZE
rhs = [0]*10000
lb = [-1]*10000
ub = [1]*10000
vtype = [GRB.CONTINUOUS]*10000
A = (np.column_stack((x, -np.ones(10000)))).T
sense = [GRB.EQUAL]*11
solution = [0]*10000
lp_optimize(rows, cols, c, obj_sense, A, sense, rhs, lb, ub, vtype,
                   solution)

Optimize a model with 11 rows, 10000 columns and 108565 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [6e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.05s
Presolved: 11 rows, 10000 columns, 108565 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 5.500e+01
 Factor NZ  : 6.600e+01 (roughly 4 MBytes of memory)
 Factor Ops : 5.060e+02 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.60449233e+05  4.20598370e+07  2.23e+04 2.84e-14  3.15e+03     0s
   1   4.46315274e+01  4.83933163e+06  5.92e-09 6.25e-13  2.42e+02     0s
   2   5.28051237e+02  1.75809434e+05  6.55e-11 6.68e-13  8.76e+00     0s
   3   9.99472699e+02  2.10455105e+03  2.04e-10 6.49e-13  5.53e-02     0s
   4 

True

# $$ 3. \quad Conclusion:$$
    In part(b), dual(0.19s) is faster than the primal(0.22s)